# Extraction des données complètes pour le Rhones

In [41]:
import requests
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
import time

# URL de base de l'API
base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe03existant/lines"

In [42]:
# Chargement du ficher des adresses du rhones.
df69 = pd.read_csv("data/adresses-69.csv", sep=";", low_memory=False)

In [43]:
# Extraction des codes postaux.
code_postaux = df69["code_postal"].unique()

# Methode de requête de l'API

In [44]:
# Liste des codes qui n'ont pas pu être récupérés.
missed_codes = list()


def query_api(
    params: dict,
    base_url: str = base_url,
    next_url: str | None = None,
    retries: int = 3,
    backoff: int = 2,
) -> list[dict]:
    """Récupère toutes les pages d'une requête API en boucle avec gestion des erreurs."""

    all_results = []

    recuperated = 0

    # Recuperation du code postal des paramètres.
    postal_code = params["qs"].split(":")[-1]

    while True:
        for attempt in range(retries):
            try:
                if next_url:
                    response = requests.get(url=next_url)
                else:
                    response = requests.get(url=base_url, params=params)

                if response.status_code == 200:
                    content = response.json()

                    # Monitoring
                    recuperated += (
                        (content["total"] - recuperated)
                        if (content["total"] - recuperated) < params["size"]
                        else params["size"]
                    )

                    if content["total"] == 0:
                        print("No data found from the query.", flush=True)
                        return []

                    all_results.extend(content["results"])
                    print(
                        f"{postal_code}: {recuperated} éléments sur {content['total']} ({int(recuperated * 100 / content['total'])}%)",
                        flush=True,
                    )

                    next_url = content.get("next")
                    break  # sortie du retry loop si succès

                # Gestion d'erreur si requête invalide.
                else:
                    print(
                        f"{postal_code}: Erreur de connexion avec l'API!, tentative {attempt + 1}/{retries}",
                        flush=True,
                    )
                    time.sleep(backoff * (attempt + 1))  # backoff progressif

            # Gestion d'erreur si problème de connexion.
            except requests.RequestException as e:
                print(
                    f"{postal_code}: Erreur réseau: {e}, tentative {attempt + 1}/{retries}",
                    flush=True,
                )
                time.sleep(backoff * (attempt + 1))  # backoff progressif

        # else de boucle for (si trop d'échecs.)
        else:
            print(f"{postal_code}: Échec permanent, abandon de cette requête.")

            # Addition du code manqué dans le dictionnaire.
            missed_codes.append(postal_code)

            return all_results  # Retourner les informations récupérés jusqu'à présent.

        if not next_url:
            break

    return all_results

## Extraction normale

In [45]:
df_list = list()

for i, code in enumerate(code_postaux):
    print()
    print(f"==Récupération codes postaux: {i + 1}/{len(code_postaux)}==")

    params = {"size": 10000, "qs": f"code_postal_ban:{code}"}
    df_list.extend(query_api(params=params, retries=5))

# Concaténation de la liste de dictionnaire en dataframe.
df = pd.DataFrame(df_list)

df.to_csv("existants_69.csv")


==Récupération codes postaux: 1/91==
69790: 197 éléments sur 197 (100%)

==Récupération codes postaux: 2/91==
69170: 2893 éléments sur 2893 (100%)

==Récupération codes postaux: 3/91==
69250: 3167 éléments sur 3167 (100%)

==Récupération codes postaux: 4/91==
69380: 3216 éléments sur 3216 (100%)

==Récupération codes postaux: 5/91==
69009: 10000 éléments sur 21052 (47%)
69009: 20000 éléments sur 21052 (95%)
69009: 21052 éléments sur 21052 (100%)

==Récupération codes postaux: 6/91==
69008: 10000 éléments sur 24960 (40%)


KeyboardInterrupt: 

## Multitreading code

Le code finit en erreur, car trop de requêtes simultannées sont demandées à l'API (l'API coupe la connection).

In [ ]:
# def fetch_for_code(code):
#     """Exécute query_api pour un code postal donné."""
#     params = {
#         "size": 750,
#         "page": 1,
#         "qs": f"code_postal_ban:{code}"
#     }
#     return query_api(params=params, retries=5)


# # Liste des DataFrames pour tous les codes postaux
# with ThreadPoolExecutor(max_workers=10) as executor:
#     all_lists = list(executor.map(fetch_for_code, code_postaux))

# list_to_df = []
# for sublist in all_lists:
#     list_to_df.extend(sublist)

# # Concaténation finale
# df_final = pd.DataFrame(list_to_df)

# # Ecriture.
# df_final.to_csv("existants_69.csv")


In [ ]:
# # Vérification de l'extraction.
# df = pd.read_csv("existants_69.csv", index_col=0)

# Batiment neuf

Utiliser l'URL suivante: https://data.ademe.fr/data-fair/api/v1/datasets/dpe02neuf/lines

In [9]:
# Récupération des logements neufs.
new_habitation_list = list()

base_url = "https://data.ademe.fr/data-fair/api/v1/datasets/dpe02neuf/lines"

for i, code in enumerate(code_postaux):
    print()
    print(f"==Récupération codes postaux: {i + 1}/{len(code_postaux)}==")

    params = {"size": 1500, "qs": f"code_postal_ban:{code}"}

    new_habitation_list.extend(query_api(params=params))

# Concaténation de la liste de dictionnaire en dataframe.
df_news = pd.DataFrame(df_list)

# Ecriture du fichier en csv.
df_news.to_csv("neufs_69.csv")


==Récupération codes postaux: 1/91==
69790: 197 éléments sur 197 (100%)

==Récupération codes postaux: 2/91==
69170: 1500 éléments sur 2893 (51%)


KeyboardInterrupt: 

# Concaténation des bâtiments existants et neufs.

In [9]:
# Relecture des fichiers csv si nécessaires.
df1 = pd.read_csv("existants_69.csv")

/var/folders/vm/277zr2vx0wv9k3k3scsf5mh00000gn/T/ipykernel_6099/609199760.py:2: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv("existants_69.csv")


In [10]:
# neufs
df2 = pd.read_csv("neufs_69.csv")

/var/folders/vm/277zr2vx0wv9k3k3scsf5mh00000gn/T/ipykernel_6099/2481822624.py:2: DtypeWarning: Columns (50,144,146,147,150,152,153,155,156,157,158,159,160,162,163,164,165,166,167,168,170,172,173,174,175,176,178,179,181,182,183) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv("neufs_69.csv")


In [11]:
# Effectuer la concaténation des deux dataframes.
df = pd.concat([df1, df2], join="inner")

In [20]:
# Exports du resultat final.
df.to_csv("logements_69.csv")

# Extraction des données d'ENEDIS de consommation par adresse.

In [12]:
# Ouverture du fichier d'ENEDIS.
df_enedis = pd.read_csv(
    "data/consommation-annuelle-residentielle-par-adresse-69008.csv", sep=";"
)

FileNotFoundError: [Errno 2] No such file or directory: 'data/consommation-annuelle-residentielle-par-adresse-69008.csv'

In [25]:
df_enedis

,Unnamed: 0,annee,code_iris,nom_iris,numero_de_voie,indice_de_repetition,type_de_voie,libelle_de_voie,code_commune,nom_commune,segment_de_client,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,adresse,code_epci,code_departement,code_region,tri_des_adresses
0,66969,2022,693880202,Les Alouettes-Bachut,54.0,B,RUE,SAINT MATHIEU,69123,Lyon,RESIDENTIEL,17,109.700,6.453,2.771,54 B RUE SAINT MATHIEU,200046977.0,69.0,84.0,170031
1,66978,2022,693880202,Les Alouettes-Bachut,49.0,NaN,RUE,SAINT NESTOR,69123,Lyon,RESIDENTIEL,25,95.018,3.801,2.771,49 RUE SAINT NESTOR,200046977.0,69.0,84.0,170101
2,66979,2022,693880102,Colbert,3.0,NaN,RUE,SAINT NESTOR,69123,Lyon,RESIDENTIEL,123,244.940,1.991,2.771,3 RUE SAINT NESTOR,200046977.0,69.0,84.0,170106
3,66980,2022,693880104,Marius Berliet Nord,26.0,NaN,RUE,SAINT NESTOR,69123,Lyon,RESIDENTIEL,35,164.746,4.707,2.771,26 RUE SAINT NESTOR,200046977.0,69.0,84.0,170109
4,66981,2022,693880101,Jean Moulin,2.0,NaN,RUE,SAINT NESTOR,69123,Lyon,RESIDENTIEL,11,21.472,1.952,2.771,2 RUE SAINT NESTOR,200046977.0,69.0,84.0,170111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9307,2368625,2023,693880204,Montplaisir Sud,10.0,NaN,PROMENADE,L ET N BULLUKIAN,69123,Lyon,RESIDENTIEL,56,65.831,1.176,2.654,10 PROMENADE L ET N BULLUKIAN,200046977.0,69.0,84.0,169802
9308,2368636,2023,693880302,Rockefeller-La Buire,127.0,NaN,RUE,LAENNEC,69123,Lyon,RESIDENTIEL,16,25.171,1.573,2.654,127 RUE LAENNEC,200046977.0,69.0,84.0,169886
9309,2368637,2023,693880302,Rockefeller-La Buire,123.0,NaN,RUE,LAENNEC,69123,Lyon,RESIDENTIEL,14,31.795,2.271,2.654,123 RUE LAENNEC,200046977.0,69.0,84.0,169888
9310,2368638,2023,693880302,Rockefeller-La Buire,113.0,NaN,RUE,LAENNEC,69123,Lyon,RESIDENTIEL,13,25.944,1.996,2.654,113 RUE LAENNEC,200046977.0,69.0,84.0,169893


In [ ]:
# Recuperation des codes IRIS.

In [ ]:
# Utilisation de l'API d'ENEDIS
def query_enedis_api(
    params: dict, base_url: str = base_url, retries: int = 3, backoff: int = 2
) -> list[dict]:
    """Récupère toutes les pages d'une requête API en boucle avec gestion des erreurs."""

    # Create a copy of the parameters.
    params_local = params.copy()
    all_results = []

    recuperated = 0
    n = params_local["limit"]
    total = None
    params_local["offset"] = 0

    while True:
        for attempt in range(retries):
            try:
                response = requests.get(url=base_url, params=params_local)

                if response.status_code == 200:
                    content = response.json()

                    # Check if the query have found data.
                    if content["total_count"] == 0:
                        print("No data found from the query.", flush=True)
                        return []

                    # Monitoring.

                    # Update the count of total and recuperated.
                    if not total:
                        total = content["total_count"]

                    total = total - n if total - n > 0 else 0
                    recuperated = (
                        recuperated + n if total - n > 0 else recuperated + total
                    )

                    # Add the result to the list.
                    all_results.extend(content["results"])

                    print(
                        f"{recuperated}/{content['total_count']} éléments récupérés ({int(recuperated * 100 / content['total_count'])}%)",
                        flush=True,
                    )

                    # Rajotuer l'offset
                    params_local["offset"] += n

                    break  # sortie du retry loop si succès

                # Gestion d'erreur si requête invalide.
                else:
                    print(
                        f"Erreur de connexion avec l'API!, tentative {attempt + 1}/{retries}",
                        flush=True,
                    )
                    time.sleep(backoff * (attempt + 1))  # backoff progressif

            # Gestion d'erreur si problème de connexion.
            except requests.RequestException as e:
                print(
                    f"Erreur réseau: {e}, tentative {attempt + 1}/{retries}", flush=True
                )
                time.sleep(backoff * (attempt + 1))  # backoff progressif

        # else de boucle for (si trop d'échecs.)
        else:
            print("Échec permanent, abandon de cette requête.")

            return all_results  # Retourner les informations récupérés jusqu'à présent.

        # arrêt de la boucle si tout a été extrait.
        if total == 0:
            break

        # endfor

    # endwhile

    return all_results

In [36]:
base_url = "https://data.enedis.fr/api/explore/v2.1/"
dataset = "consommation-annuelle-residentielle-par-adresse"
url = base_url + f"catalog/datasets/{dataset}/records"

In [ ]:
# # Test de requete
# response = requests.get(
#     url=url,
#     params={
#         "limit": 100,
#         "where": f'code_commune=69006 and nom_commune like "Amplepuis"',
#     },
# )

# if response.status_code == 200:
#     content = response.json()
# else:
#     print(response)

In [ ]:
# pd.DataFrame(content["results"])


In [38]:
# Récupération des code iris  pour le 69
params = {"limit": 100, "where": f"code_departement=69", "group_by": "code_iris"}
enedis_data = query_enedis_api(base_url=url, params=params)


100/641 éléments récupérés (15%)
200/641 éléments récupérés (31%)
300/641 éléments récupérés (46%)
400/641 éléments récupérés (62%)
500/641 éléments récupérés (78%)
541/641 éléments récupérés (84%)
541/641 éléments récupérés (84%)


In [40]:
len(enedis_data)

641

In [ ]:
# Variable globalE pour récupérer les données d'enedis
enedis_data = []

# Boucler par annéede 2018 -> 2024
for annee in range(2018, 2025, 1):
    print()
    print(f"==Récupération de l'année: {annee}==")

    params = {
        "limit": 100,
        "where": f"code_departement={69}",
        "refine": f"annee:{annee}",
    }

    # Verification du nombres de lignes (< 10000)
    check_request = requests.get(url=base_url, params=params | {"limit": 0})

    enedis_data.extend(query_enedis_api(base_url=url, params=params))

In [115]:
pd.DataFrame(enedis_data)

,annee,code_iris,nom_iris,numero_de_voie,indice_de_repetition,type_de_voie,libelle_de_voie,code_commune,nom_commune,segment_de_client,nombre_de_logements,consommation_annuelle_totale_de_l_adresse_mwh,consommation_annuelle_moyenne_par_site_de_l_adresse_mwh,consommation_annuelle_moyenne_de_la_commune_mwh,adresse,code_epci,code_departement,code_region,tri_des_adresses
0,2023,691700000,Rontalon (commune non irisée),40,None,ROUTE,DE FONDRIEU,69170,Rontalon,RESIDENTIEL,19,28.373,1.493,5.578,40 ROUTE DE FONDRIEU,246900740,69,84,306339
1,2019,691700000,Rontalon (commune non irisée),40,None,ROUTE,DE FONDRIEU,69170,Rontalon,RESIDENTIEL,19,39.701,2.090,6.380,40 ROUTE DE FONDRIEU,246900740,69,84,309872
2,2020,691700000,Rontalon (commune non irisée),40,None,ROUTE,DE FONDRIEU,69170,Rontalon,RESIDENTIEL,19,38.007,2.000,6.391,40 ROUTE DE FONDRIEU,246900740,69,84,314051
3,2021,691700000,Rontalon (commune non irisée),40,None,ROUTE,DE FONDRIEU,69170,Rontalon,RESIDENTIEL,19,38.119,2.006,6.633,40 ROUTE DE FONDRIEU,246900740,69,84,314961
4,2024,691700000,Rontalon (commune non irisée),40,None,ROUTE,DE FONDRIEU,69170,Rontalon,RESIDENTIEL,19,32.556,1.713,5.657,40 ROUTE DE FONDRIEU,246900740,69,84,329504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5692,2022,692600000,Vernaison (commune non irisée),7,None,RUE,DU PERONNET,69260,Vernaison,RESIDENTIEL,24,45.285,1.887,5.492,7 RUE DU PERONNET,200046977,69,84,370615
5693,2022,692600000,Vernaison (commune non irisée),3,None,RUE,DU PERONNET,69260,Vernaison,RESIDENTIEL,21,42.657,2.031,5.492,3 RUE DU PERONNET,200046977,69,84,370617
5694,2022,691260000,Marcy (commune non irisée),125,None,ROUTE,DE FRONTENAS,69126,Marcy,RESIDENTIEL,11,79.578,7.234,8.258,125 ROUTE DE FRONTENAS,200040574,69,84,174940
5695,2024,691260000,Marcy (commune non irisée),125,None,ROUTE,DE FRONTENAS,69126,Marcy,RESIDENTIEL,12,99.043,8.254,7.696,125 ROUTE DE FRONTENAS,200040574,69,84,190848
